# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
# DONE Define a query and run
query = "Who won UEFA football player of the year last year?"
result = client.search(query)

In [5]:
result

{'query': 'Who won UEFA football player of the year last year?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.uefa.com/uefachampionsleague/news/026c-1317c5d44053-673457b2aa24-1000--jorginho-wins-uefa-men-s-player-of-the-year-award/',
   'title': "Jorginho wins UEFA Men's Player of the Year award",
   'content': "Chelsea and Italy midfielder Jorginho has been named 2021/21 UEFA Men's Player of the Year. The 29-year-old, who last season became only the tenth player to",
   'score': 0.99953806,
   'raw_content': None},
  {'url': 'https://www.sportsadda.com/football/features-football/uefa-player-of-the-year-award-mens-womens-winners-list/',
   'title': 'UEFA Player of the Year Award: Get the complete list of winners',
   'content': "UEFA Player of the Year Award: Get the complete list of winners - Sportsadda 4.   UEFA Player of the Year Award: Get the complete list of winners UEFA Player of the Year Award: Get the complete list of winners 

## Define Web Search tool

In [6]:
# DONE Define the search tool
@tool
def web_search(query: str) -> Dict:
    return client.search(query) 

In [7]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [9]:
# DONE Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful agent that uses it's own knowledge to answer questions."
)

In [10]:
# DONE Agent with web search tool
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful agent that uses it's own knowledge and web search to answer questions.",
    tools=tools
)

In [11]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [12]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful agent that uses it's own knowledge to answer questions., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events or outcomes beyond October 2023, including the 2025 Oscars. You might want to check the latest news or official announcements for that information., tool_calls = None)


In [13]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information on events or outcomes beyond October 2023, including the 2025 Oscars. You might want to check the latest news or official announcements for that information.


In [14]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful agent that uses it's own knowledge to answer questions., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events or outcomes beyond October 2023, including the 2025 Oscars. You might want to check the latest news or official announcements for that information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2023, several key developments in AI technology were noteworthy:

1. **Generative AI Advancements**: Generative models, particularly in the realm of text and im

In [15]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2023, several key developments in AI technology were noteworthy:

1. **Generative AI Advancements**: Generative models, particularly in the realm of text and image generation, continued to improve. Models like OpenAI's GPT-4 and DALL-E showed significant advancements in generating coherent text and high-quality images, respectively.

2. **AI in Healthcare**: AI applications in healthcare were expanding, particularly in diagnostics, personalized medicine, and drug discovery. AI systems were increasingly used to analyze medical images, predict patient outcomes, and assist in clinical decision-making.

3. **Natural Language Processing (NLP)**: NLP technologies became more sophisticated, with improvements in understanding context, sentiment analysis, and multilingual capabilities. AI chatbots and virtual assistants were becoming more effective in customer service and personal assistance.

4. **AI Ethics and Regulation**: Discussions around AI ethic

**Web Agent**

In [16]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful agent that uses it's own knowledge and web search to answer questions., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_wzfXzWESRrcuONyIXntJpRmI', function=Function(arguments='{"query":"2025 Oscar for International Movie winner"}', name='web_search'), type='function')])
 -> (role = tool, content = "{'query': '2025 Oscar for International Movie winner', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://people.com/oscars-2025-im-still-here-brazil-best-international-feature

In [17]:
print(run1.get_final_state()["messages"][-1].content)

The winner of the 2025 Oscar for Best International Feature was Brazil's film **"I'm Still Here."** This award was announced during the Academy Awards held on March 2, 2025. The film is set in 1970s Rio de Janeiro during a period of military dictatorship and is based on a true story. 

For more details, you can check the sources: [People](https://people.com/oscars-2025-im-still-here-brazil-best-international-feature-11684447) and [AP News](https://apnews.com/article/best-international-film-2025-oscars-f01a852a3e3e53bbc6f3d667a484f8df).


In [19]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology in 2025?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful agent that uses it's own knowledge and web search to answer questions., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_wzfXzWESRrcuONyIXntJpRmI', function=Function(arguments='{"query":"2025 Oscar for International Movie winner"}', name='web_search'), type='function')])
 -> (role = tool, content = "{'query': '2025 Oscar for International Movie winner', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://people.com/oscars-2025-im-still-here-brazil-best-international-feature

In [20]:
print(run2.get_final_state()["messages"][-1].content)

As of 2025, several significant developments and trends in AI technology are emerging:

1. **AI-Powered Agents**: AI is evolving from a simple tool to an integral part of daily life and work. AI-powered agents are becoming more autonomous, capable of handling complex tasks and simplifying processes for users.

2. **RAPTOR System**: Researchers at Purdue University have introduced RAPTOR, an AI-powered defect-detection system that combines high-resolution X-ray imaging with machine learning. This innovation is aimed at improving quality control in manufacturing and other industries.

3. **Multimodal AI**: This trend focuses on the integration of various types of data (text, images, audio) to enhance context understanding and improve AI interactions. Multimodal AI systems are becoming more sophisticated, enabling richer and more intuitive user experiences.

4. **AI Reasoning and Frontier Models**: There is a growing emphasis on AI reasoning capabilities, allowing systems to not only proc

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow